# Coder 에이전트 구축

Navigator 에이전트가 브라우저를 탐색하여 지도를 그리는 역할이었다면, 이번에 만들 **Coder 에이전트**는 파이썬 코드를 직접 작성하고 실행하며 디버깅하는 '개발자' 역할을 수행합니다.

단순히 코드를 짜서 글자로만 알려주는 LLM이 아니라,
**코드를 [.py](cci:7://file:///d:/agent_research/github-repos/AAWS_project/app/client.py:0:0-0:0) 파일로 저장하고 → 직접 실행해보고 → 에러가 나면 스스로 고치는(디버깅)** 능력을 갖춘 완벽한 자율 에이전트입니다.

### 핵심 아이디어 1: Python Execution Tool
에이전트에게 "코드를 실행하는 능력"을 주기 위해 `subprocess` 모듈을 활용하여 지정된 격리 공간(`code_artifacts/`) 안에서만 파이썬 스크립트를 작성하고 실행하는 전용 도구를 만들어 제공합니다.

### 핵심 아이디어 2: File Search Middleware (에이전트의 시야 확장)
개발자가 코드를 짜려면 현재 작업 폴더에 어떤 파일들이 있는지 파악해야 합니다.
LangChain의 `FilesystemFileSearchMiddleware`를 추가해, 에이전트가 `code_artifacts/` 폴더 내의 파일들을 검색하고 읽어볼 수 있는 도구를 자동으로 쥐여줍니다.


In [3]:
import os, sys
from dotenv import load_dotenv

load_dotenv(override=True)
#load_dotenv("/mnt/d/agent_research/github-repos/home/AAWS_project/.env", override=True)

# PROJECT_ROOT를 .env에서 읽기 (없으면 현재 디렉토리)
project_root = os.getenv("PROJECT_ROOT", os.getcwd())

# 프로젝트 루트가 유효하지 않으면, 현재 위치에서 상위로 찾기
if not os.path.exists(os.path.join(project_root, "app")):
    # 상위 디렉토리 탐색
    current = os.getcwd()
    for _ in range(5):
        if os.path.exists(os.path.join(current, "app")):
            project_root = current
            break
        current = os.path.dirname(current)

# Working Directory 설정
os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import app

print(f"✅ Project Root: {project_root}")
print(f"✅ Working Directory: {os.getcwd()}")

In [50]:
import os
import subprocess
from langchain.tools import tool

# ==========================================
# 1. 🛠️ 파이썬 코드 실행 공간 및 도구 (Tool)
# ==========================================

# 작업 및 실행 파일들이 모일 대상 디렉토리 (없으면 자동 생성)
ARTIFACT_DIR = os.path.join(os.getenv("PROJECT_ROOT", os.getcwd()), "code_artifacts")
os.makedirs(ARTIFACT_DIR, exist_ok=True)

@tool(parse_docstring=True)
def execute_python_code(code: str, filename: str = "generated_script.py") -> str:
    """주어진 파이썬 코드를 로컬 환경의 파일로 저장하고 실행한 뒤, 그 결과(표준 출력 및 에러)를 반환합니다.
    코드가 정상 작동하는지 테스트하고 디버깅할 때 사용하세요.
    
    Args:
        code: 실행할 완전한 파이썬 스크립트 코드 내용 (모든 import 포함 필수).
        filename: 코드를 저장할 파이썬 파일명 (기본값: 'generated_script.py').
    """
    # ✅ 항상 code_artifacts 경로 내부로 저장되도록 경로 강제 처리
    safe_filename = os.path.basename(filename)
    filepath = os.path.join(ARTIFACT_DIR, safe_filename)
    
    print(f"\n🐍 [Coder Tool] '{filepath}' 파일 생성 및 실행 중...")
    
    try:
        # 코드를 파일로 저장 (무조건 덮어쓰기)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(code)
            
        # 파이썬 실행 (작업 디렉토리를 ARTIFACT_DIR 내부로 한정)
        result = subprocess.run(
            ["python", safe_filename], 
            cwd=ARTIFACT_DIR, # ✅ 작업 디렉토리 지정!
            capture_output=True, 
            text=True, 
            timeout=30 # 무한 루프 등 시간끌기 방지
        )
        
        output = result.stdout
        if result.stderr:
            output += f"\n[Error Output]\n{result.stderr}"
            
        if not output.strip():
            output = "[System] 코드가 에러 없이 실행되었으나 출력된 내용이 없습니다."
            
        return output
        
    except subprocess.TimeoutExpired:
        return "[Error] 실행 시간(30초)을 초과했습니다. 무한 루프 수정을 시도하세요."
    except Exception as e:
        return f"[System Error] 코드 실행 오류 발생: {str(e)}"


### Coder 에이전트(Agent) 조립하기
LangChain의 [create_agent]를 활용해 파이썬 실행 도구와 파일 검색 미들웨어를 장착한 에이전트를 선언합니다.


In [51]:
from dataclasses import dataclass
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import FilesystemFileSearchMiddleware

# ==========================================
# 2. 🤖 Coder 에이전트 정의 (create_agent)
# ==========================================
@dataclass
class CoderContext:
    pass

coder_model = init_chat_model("google_genai:gemini-2.0-flash", temperature=0.2)
checkpointer = InMemorySaver()

CODER_SYSTEM_PROMPT = f"""
당신은 파이썬 코드를 작성, 분석, 실행할 수 있는 '시니어 파이썬 개발자'입니다.

[역할 및 지침]
1. 목적에 맞게 도구를 명확히 구분해서 사용하세요:
   - 파일 내용을 읽거나 검색할 때: 파이썬 코드를 작성하지 말고, 반드시 내장된 파일 검색 도구를 우선적으로 사용하세요.
   - 새로운 로직이나 파이썬 스크립트를 작성하여 테스트할 때: 코드를 작성한 후 `execute_python_code` 도구를 사용하세요.
2. 코드를 작성했다면 반드시 `execute_python_code`를 실행하여 결과를 검증하세요.
3. 실행 로그에 에러(Error)가 발생하면, 즉시 에러 사유를 파악하고 코드를 수정한 뒤 다시 실행(디버깅)하세요. 에러 없이 성공할 때까지 스스로 반복해야 합니다.
4. 모든 작업이 완료되면, 최종적으로 해결된 방법과 결과를 사용자에게 짧고 명확하게 요약해 주세요.
"""

coder_agent = create_agent(
    model=coder_model,
    system_prompt=CODER_SYSTEM_PROMPT,
    context_schema=CoderContext,
    tools=[execute_python_code],
    middleware=[
        FilesystemFileSearchMiddleware(
            root_path=ARTIFACT_DIR, # ✅ 검색 범위를 code_artifacts로 강제
            use_ripgrep=True, 
            max_file_size_mb=10,
        )
    ],
    checkpointer=checkpointer
)


In [45]:
ARTIFACT_DIR

In [52]:
from langchain_core.messages import HumanMessage

# ==========================================
# 3. 🚀 테스트: 파일 쓰기/읽기 및 실행 확인
# ==========================================

config = {"configurable": {"thread_id": "coder_session_0"}}
context = CoderContext()

query1 = "파이썬 코드를 작성해서 실행해줘. 내용은 'Hello World' 라는 텍스트가 담긴 'test.txt' 파일을 생성하는 거야."
print(f"👤 User: {query1}\n")
print("-" * 50)

response1 = await coder_agent.ainvoke(
    {"messages": [HumanMessage(query1)]},
    config=config, context=context
)
print(f"\n🤖 Assistant: {response1['messages'][-1].content}\n")
print("=" * 80)


In [53]:
for m in response1["messages"]:
    m.pretty_print()

In [54]:
    query2 = "좋아, 그럼 방금 생성한 'test.txt' 파일 안에 무슨 내용이 적혀있는지 정확히 확인하고 알려줄래?"
    print(f"👤 User: {query2}\n")
    print("-" * 50)
    
    response2 = await coder_agent.ainvoke(
        {"messages": [HumanMessage(query2)]},
        config=config, context=context
    )
    print(f"\n🤖 Assistant: {response2['messages'][-1].content}")

In [55]:
for m in response2["messages"]:
    m.pretty_print()

Coder 에이전트에게 1부터 100까지 소수(Prime number)를 구하는 코드를 짜달라고 요청해 보겠습니다. 
과연 직접 계산해서 답을 내는지, 아니면 코드를 짜서 실행시켜 보는지 확인하세요!


In [58]:
config = {"configurable": {"thread_id": "coder_session_1"}}
context = CoderContext()

query = "1부터 100까지의 숫자 중에서 소수(Prime Number)만 찾아내는 파이썬 코드를 작성하고 실행해서, 나에게 출력 결과만 깔끔하게 보여줘."
print(f"👤 User: {query}\n")
print("-" * 50)

response3 = await coder_agent.ainvoke(
    {"messages": [HumanMessage(query)]},
    config=config,
    context=context
)

print("-" * 50)
print(f"\n🤖 Assistant: {response['messages'][-1].content}")

In [59]:
for m in response3["messages"]:
    m.pretty_print()

### 실습: Coder 에이전트 앱 서버 연동하기

이제 똑똑한 코딩 비서도 완성되었으니, 기존의 앱 서버(`app`)에 **Coder 에이전트**를 탑재하여 채팅 화면에서 편하게 디버깅을 시켜봅시다! Navigator 에이전트를 추가했을 때와 비슷한 단계를 거칩니다.
